In [28]:
%load_ext autoreload

%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import builtins

# I'm not sure why this is needed, but this resolves a runtime crash when run from the command line
# reassign the builtins.print function to bprint
bprint = builtins.print

try:
    from . import constants
except ImportError:
    import constants

try:
    from . import error_msgs
except ImportError:
    import error_msgs

try:
    from .filestream import GoogleDrivePath, GDStudentPath
except ImportError:
    from filestream import GoogleDrivePath, GDStudentPath
    
import logging
from logging import handlers
from logging import config

logging.config.fileConfig(constants.LOGGING_CONFIG, defaults={'logfile': constants.LOG_FILE} )


import sys
from pathlib import Path
import textwrap
from datetime import datetime
import re
import subprocess
import shlex
import shutil

In [30]:
import PySimpleGUI as sg
import ArgConfigParse
from tinydb import TinyDB, Query


In [31]:
# FORMAT = constants.FORMAT
# DATEFMT = constants.DATEFMT
# logging.basicConfig(format=FORMAT, datefmt=DATEFMT,
#                     level=logging.DEBUG)



In [32]:
def do_exit(e='unknown error in unknown module!', exit_status=99):
    '''handle exits and return exit function with either a soft_exit or hard_exit -
        The returned function can be executed by the calling process when it is ready 
        rather than forcing an exit immediately 
    
        soft_exit prints message and optionally logs message
        hard_exit prints message, logs and calls sys.exit(exit_status)
    
    Args:
        e(`str`): error or message detailing reason for exiting
        exit_status(int): exit value --
            0: soft_exit with no logging -- normal exit with no issues
            1: soft_exit with logging -- exit due to recoverable issue
            >1: hard_exit with logging -- abort execution with sys.exit(exit_status)
            
    Returns:
        function: soft_exit, hard_exit'''
    help_msg = f'try:\n{sys.argv[0]} -h for help'
    def hard_exit():
        print(e)
        sys.exit(exit_status)
        
    def soft_exit():
        print(e)
        return(e)
    
    if exit_status > 1:
        logging.error(f'fatal error:\n\t{e}')
        return(hard_exit)
    
    if exit_status == 1:
        logging.warning(f'exited before completion with code {exit_status}')
        logging.warning(e)
        print(help_msg)
        return(soft_exit)
    
    if exit_status < 1:
        logging.debug(e)
        return(soft_exit)

In [33]:
def wrap_print(t='', width=None, supress_print=False):
    '''print a text-wrapped string
    
    Args:
        t(`str`): text to wrap
        width(`int`): characters to wrap -- defaults to constants.TEXT_WIDTH
        
    Returns:
        str'''
    if not width:
        width = constants.TEXT_WIDTH
        
    wrapper = textwrap.TextWrapper(width=width, break_long_words=False, replace_whitespace=False)
    result = '\n'.join([wrapper.fill(line) for line in t.splitlines()])
# this causes a runtime crash; it's unclear why, but is resolved by reassigning bprint = builtins.print 
#     builtins.print(result)
    if not supress_print:
        bprint(result)
    return result

In [34]:
def parse_cmdargs():
    args = ArgConfigParse.CmdArgs()
    
    args.add_argument('-i', '--insert_source', ignore_none=True, 
                      metavar='/path/to/student/records/',
                      type=str, dest='insert_source', help='Full path to file to be inserted')
    
    args.add_argument('-g', '--google_drive', ignore_none=True, 
                      metavar='/Volumes/GoogleDrive/Shared drives/ASH Cum Folders/folder/',
                      type=str, dest='main__drive_path', help='Full path to Google Drive Shared Drive containing cumulative files')

    args.add_argument('-l', '--log_level', ignore_none=True, metavar='ERROR, WARNING, INFO, DEBUG', 
                      type=str, dest='main__log_level', help='Logging level -- Default: WARNING')
    args.add_argument('-v', '--version', dest='version', action='store_true',
                      default=False, help='Print version number and exit')
    
    args.add_argument('-u', '--update_drive', action='store_true',
                       default=False, dest ='update_drive', help='Update config file with supplyed -g option')
    
    args.add_argument('--more_help', dest='more_help', action='store_true',
                       default=False, help='Print extened help and exit')


    args.parse_args()
    return args.nested_opts_dict
    

In [35]:
def read_config(files):
    '''parse .ini files 
    
    Args:
        files(`list`): list of `str` containing files in .ini format to parse
    
    Returns:
        `dict`: nested dict of configuration'''
    parser = ArgConfigParse.ConfigFile(config_files=files, ignore_missing=True)
    parser.parse_config()
    
    return parser.config_dict

In [36]:
def check_drive_path(drive_path=None):
    '''check that path is a valid google drive path and contains the appropriate sentry file
    
    Args:
        drive_path(`str`): path to google drive containg cummulative folders and sentry file
    
    Retruns:
        `tuple` of `bool`, `str`: When true, drive is OK; when false, drive is not valid; str contains errors'''
    # this is super redundant -- checks the following:
    # * is a path
    # * is a google drive path
    # * if sentry file exists
    # this may be a good idea considering how some users have run into many problems with this
    SENTRY_FILE = constants.SENTRY_FILE    
    sentry_file_path = drive_path/Path(SENTRY_FILE)
    drive_ok = True
    msg = None
    
    if not drive_path:
        logging.info('no google drive specified')
        drive_ok = False
        msg = 'No Google Drive specified'
        return drive_ok, msg
    else:
        drive_path = Path(drive_path)
    
    if not drive_path.exists():
        logging.warning(f'specified path "{drive_path}" does not exist')
        drive_ok = False
#         msg = f'The Google Drive "{drive_path}" does not appear to exist on Google Drive'
        msg = error_msgs.PATH_ERROR.format(drive_path=drive_path)
        return drive_ok, msg
    else:
        google_drive = GoogleDrivePath(drive_path)
    
    try:
        google_drive.get_xattr('user.drive.id')
    except ChildProcessError as e:
        logging.warning(f'specified path "{drive_path}" is not a Google Drive path')
#         msg = f'The Google Drive "{drive_path}" does not appear to be a valid google Shared Drive'
        msg = error_msgs.NON_GDRIVE_ERROR.format(drive_path=drive_path)
        drive_ok = False
        return drive_ok, msg

 
    if not sentry_file_path.is_file():
        logging.warning(f'sentry file is missing in specified path "{drive_path}"')
        msg = error_msgs.SENTRY_ERROR.format(drive_path=drive_path, sentry_file=SENTRY_FILE)
        drive_ok = False
        
    
    
    
    return drive_ok, msg

In [37]:
def print_help():
    print('help!')

In [38]:
def window_drive_path():
    drive_path = sg.popup_get_folder('Choose the Google Shared Drive **AND** folder that contains student cumulative folders.', 
                                     title='Select A Shared Drive', 
                                     initial_folder='/Volumes/GoogleDrive/',
                                     keep_on_top=True, font=constants.FONT, 
                                     location=constants.POPUP_LOCATION)
    if drive_path:
        drive_path=Path(drive_path)
        logging.debug(f'user selected: {drive_path}')
    else:
        drive_path = None
        logging.info('no drive path selected by user')
    return drive_path

In [39]:
def init_db():
    db_file = Path(constants.STORAGE/constants.DATABASE)
    
    if not db_file.parent.exists():
        try:
            db_file.parent.mkdir(parents=True, exist_ok=True)
        except Exception as e:
            do_exit(f'failed to create DB Directory -- fatal error: {e}')
    
    return TinyDB(db_file)
    

In [40]:
# def window_get_dir():
#     file_list = None
#     glob_path = sg.popup_get_folder('Choose a folder containing files to insert into Google Shared Drive',
#                                    title='Choose a folder',
#                                    initial_folder = '~/',
#                                    keep_on_top=True,
#                                    font=constants.FONT,
#                                    location=constants.POPUP_LOCATION)
    
#     if glob_path:
#         file_list = [f for f in Path(glob_path).glob('*')]
#     else:
#         logging.info('no folder selected by user')
        
    
        
    
    
#     return file_list

In [41]:
def window_get_dir():
    file_list = None
    layout = [ [sg.Text('Choose a folder containing files to insert into Cumulative Folders',
                        font=f'{constants.FONT_FACE} {constants.FONT_SIZE+2}')],
               [sg.Text(wrap_print(f'{constants.APP_NAME} will insert the selected files into a Cumulative Folders'), font=constants.FONT)],
               [sg.Input(key='-INSERT-', font=constants.FONT), sg.FolderBrowse(font=constants.FONT, target='-INSERT-')],
               [sg.Button('OK', font=constants.FONT), sg.Button('Cancel', font=constants.FONT), ]
             ]
    
    window = sg.Window('Choose a Folder', layout, keep_on_top=True, location=constants.POPUP_LOCATION)
    
    while True:
        event, values = window.read()
        
        if event in (sg.WIN_CLOSED, 'Cancel'):
            event = 'Cancel'
            break
        if event == 'OK':
            break
            
    if values['-INSERT-']:
        file_list = [f for f in Path(values['-INSERT-']).glob('*')]
    
    window.Close()
    return event, file_list
            

In [42]:
def get_grade_level():
    grade_level = None
    width = 45
    layout = [ [sg.Text('Choose A Grade Level', font=constants.FONT)],
               [sg.Text(wrap_print(f'{constants.APP_NAME} will insert the chosen files into this grade-level folder for each student', width, True), 
                                  font=f'{constants.FONT_FACE} {constants.FONT_SIZE-2}')],
               [sg.Listbox(values=constants.STUDENT_DIRS, font=constants.FONT, size=(width-15, len(constants.STUDENT_DIRS)), key='-LIST-', enable_events=True)],
               [sg.Button('OK'), sg.Button('Cancel')]
             ]
    window = sg.Window('Choose a Grade Level', layout, keep_on_top=True, location=constants.POPUP_LOCATION)
    
    while True:
        event, values = window.read()
        logging.debug(f'user slected: {values}')
        
        if event in (sg.WIN_CLOSED, 'Cancel'):
            break
        if event == 'OK':
            grade_level = values['-LIST-'][0]
            break
    window.close()
    return event, grade_level

In [43]:
def sort_files(files):
    '''sort list of Path() file objects in to a "good" and "bad" set
        "good" files contain 6 digit substring (likely PowerSchool SIS student number)
        "bad" not files, contain fewer than 6 digit substring, contain multiple 6 digit substring
        
    Args:
        files(`list`): list of Path() objects
    
    Returns:
        (good_files(`dict`), bad_files(`dict`))
    
    '''
    matches = {}
    good_files = {}
    bad_files = {}
    
    for each in files:
        if each.is_file() or each.is_dir():
            matches[each] = re.match('^.*?\D?(\d{6})\D+(\d{6})?.*$', each.name)
        else:
            bad_files[each] = 'is not a file or directory'


    for key, value in matches.items():
        try:
            # there should be only one match that appears to be a student number
            # if the second match turns up 
            if value.group(2):
                bad_files[key] = 'contains too many 6 digit numbers'
            else:
                good_files[key] = value.group(1)

        except Exception:
            bad_files[key] = 'contains no 6 digit number'
    return(good_files, bad_files)

In [44]:
def cache_dirs(google_drive_path):
    
    def byte_to_list(byte): return [l for l in cache.decode('utf-8').split('\n')]
    
    command = shlex.split(f'find "{str(google_drive_path)}" -maxdepth 2 -type d')
    process = subprocess.Popen(command, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    cache, errors = process.communicate()
    
    return byte_to_list(cache), byte_to_list(errors)


In [45]:
def find_dirs(files, cache):
    
    matches = {}
    for file, student_number in files.items():
        matches[file] = [i for i in cache if student_number in i]
    
    return matches

In [46]:
def insert_files(matches, grade_level):
    failure = {}
    success = {}
    for file, match in matches.items():
        if len(match) < 1:
            failure[file] = {'msg': 'no folder exists for this student on google drive', 'matches': []}
            continue

        if len(match) > 1:
            failure[file] = {'msg': 'multiple folders exist for this student on google drive', 'matches': match}
            continue

        if len(match) == 1:
            print(match)
            success[file] = True

    return(success, failure)

In [47]:
def main_program(interactive=False, window=None, update_config=False):
    logger = logging.getLogger(__name__)
    logging.info('***** run main program *****')
    logging.info(f'interactive: {interactive}')
    
    if interactive:
        print = wrap_print    
    
    USER_CONFIG_PATH = Path(constants.USER_CONFIG_PATH)
    
    logging.debug(f'checking user config: {USER_CONFIG_PATH}')

    update_user_config = not(USER_CONFIG_PATH.exists())
    
    logging.debug(f'user config will be created: {update_user_config}')
    
    cmd_args_dict = parse_cmdargs()
    cfg_files_dict = read_config([constants.CONFIG_FILE, USER_CONFIG_PATH])
    
    config = ArgConfigParse.merge_dict(cfg_files_dict, cmd_args_dict)
    
    logging.debug('processing command line options')
    
    if config['__cmd_line']['version']:
        logging.debug('display version and exit')
        return do_exit(version_info, 0)
    
    if config['__cmd_line']['more_help'] and not interactive:
        logging.debug('display help and exit')
        print_help()
        return do_exit(' ', 0)
    
    if config['__cmd_line']['update_drive']:
        logging.debug('updating user configuration file with new google drive path')
        update_user_config = True
    
    if not config['main']['drive_path']:
        if interactive:
            logging.debug('asking user to select shared drive & cum folder')
            print('No Google Shared Drive has been set yet.')
            print('Locate the proper Google Shared Drive **and** then the `Student Cumulative Folders (AKA Student Portfolios)` folder')
            drive_path_interactive = window_drive_path()
            if not drive_path_interactive:
                return do_exit('Please choose a Google Shared Drive to proceed', 0)
            else:
                config['main']['drive_path'] = drive_path_interactive        
        if not interactive:
            return do_exit(f'Can not run without a Google Shared Drive configured.\ntry:\n{sys.argv[0]} -h for help', 1)


    
    # check that supplied path is indeed a valid cumulative folder path
    logging.debug(f"checking drive path is valid: {config['main']['drive_path']}")
    drive_path = Path(config['main']['drive_path'])
    drive_status = check_drive_path(drive_path)
    
    if window:
        window.Refresh()
    
    if not drive_status[0]:
        return do_exit(drive_status[1], 0)
    
    if interactive:
        logging.debug('asking user to select folder')
        print('Select a folder containing student files to insert')
        event, file_list = window_get_dir()
    else:
        # use command line switch 
        # file_list = [f for f in Path(path from command line here).glob('*')]
        pass
    
    logging.debug(f'event: {event}, file_list: {file_list}')        
    if file_list == None:
        return do_exit('Cannot proceed without files to insert', 0)

    # sort files into those with and without student numbers
    good_files, bad_files = sort_files(file_list)
    
    # cache all the student directories 
    cache, errors = cache_dirs(drive_path)
    
    # check the cache and bail out with some logging!
    
    # match files w/ student numbers to directories in the cache
    matches = find_dirs(good_files, cache)
    
    # ask for grade level
    while True:
        event, grade_level = get_grade_level()
        
        if grade_level:
            break
            
        if event == 'Cancel':
            logging.info('user canceled grade level selection')
            return do_exit('Processing of files canceled by user', 0)
        

    proceed = sg.PopupOKCancel(wrap_print(f'Files will be inserted into "{grade_level}"" folder for each student. This **cannot** be undone!\n\nProceed?', supress_print=True), title='Proceed?', 
                               font=constants.FONT,
                               keep_on_top=True,
                               line_width=constants.TEXT_WIDTH-10,
                               background_color="Red")

    
    if proceed == 'OK':
        pass
    else:
        return do_exit('Processing of files canceled by user', 0)
    

    # add record of those files inserted successfully into DB 
    inserted_success, inserted_failed = insert_files(matches, grade_level)

    # success

    # failure    
    
    # report back to user success and failures
    
    
    
    if update_user_config:
        logging.debug('updating user configuration file')
        try:
            logging.info(f'updating user configuration file: {USER_CONFIG_PATH}')
            ArgConfigParse.write(config, USER_CONFIG_PATH, create=True)
        except Exception as e:
            logging.warning(e)
            
    
    return do_exit('done', 0)

In [48]:
def main():
    
    logging.info(f'{constants.APP_NAME} v{constants.VERSION}')
    
    run_gui = False
    if len(sys.argv) <= 1:
        run_gui = True
    
    if '-f' in sys.argv:
        logging.debug('likely running in a jupyter environment')
        run_gui = True
    

    if run_gui:
        logging.debug('running gui')
        TEXT_WIDTH = constants.TEXT_WIDTH
        TEXT_ROWS = constants.TEXT_ROWS
        FONT_FACE = constants.FONT_FACE
        FONT_SIZE = constants.FONT_SIZE
        FONT = constants.FONT
        
        # create a wrapper that matches the text output size
        logging.debug('redefining builtin `print` to use `wrap_print`')
        print = wrap_print        

        def text_fmt(text, *args, **kwargs): return sg.Text(text, *args, **kwargs)
        layout = [ [text_fmt(f'{constants.APP_NAME}', font=f'{FONT_FACE} {FONT_SIZE+2}')],
                   [text_fmt(f'v{constants.VERSION}', font=f'{FONT_FACE} {FONT_SIZE}')],              
                   [text_fmt(f'{constants.APP_DESC}', font=f'{FONT_FACE} {FONT_SIZE}')],
                   [sg.Output(size=(TEXT_WIDTH+30, TEXT_ROWS), font=FONT)],
                   [sg.Button('Insert Files', font=FONT), sg.Button('Remove Files', font=FONT)],
                   [sg.Button('Change Shared Drive', font=FONT)],
                   [sg.Button('Help', font=FONT), sg.Button('Exit', font=FONT)]
                 ]

        window = sg.Window(f'{constants.APP_NAME}', layout=layout, keep_on_top=False,
                           location=constants.WIN_LOCATION)

        window.finalize()
        window.BringToFront()

        while True:
            (event, value) = window.read()

            if event == 'Exit' or event == sg.WIN_CLOSED:
                break
            if event == 'Help':
                pass
            if event == 'Insert Files':
                logging.debug(f'sys.argv: {sys.argv}')
                ret_val = main_program(interactive=True, window=window)
                ret_val()
                window.Refresh()
            if event == 'Remove Files':
                pass
            if event == 'Change Shared Drive':
                drive = window_drive_path()
                if drive:
                    print('')
                    print(f'Changed drive to: {drive}')
                    window.Refresh()
                    sys.argv.extend(['-g', str(drive)])
                    sys.argv.append('-u')                        
                logging.debug('run set shared drive here')
            
        window.close()

    else:
        ret_val = main_program()
        ret_val()
    
    logging.debug('done')
        


In [49]:
from IPython.core.debugger import set_trace

In [ ]:
if __name__ =='__main__':
    f = main()

2020.08.15 07:58.55 <ipython-input-48-06032514f5d6> FUNC:main - INFO:insert_files v1.0.0-devel-2020.08.11
2020.08.15 07:58.58 <ipython-input-47-c2d4b4cf3708> FUNC:main_program - INFO:***** run main program *****
2020.08.15 07:58.58 <ipython-input-47-c2d4b4cf3708> FUNC:main_program - INFO:interactive: True


In [ ]:
sys.argv

In [ ]:
if '-g' in sys.argv:
     sys.argv[sys.argv.index('-g')+1] = 'baz'
else:
    sys.argv.extend(['-g', 'baz'])

print(sys.argv)


In [ ]:
datetime.datetime().strftime()

In [ ]:
db = TinyDB('db.json')

In [ ]:
ts = int(time.time())


In [ ]:
ts = int(datetime.now().timestamp())
print(datetime.fromtimestamp(ts))

In [ ]:
sys.argv.clear()

In [ ]:
db.insert({'job_id': ts, 'files': {'foo': '/path/a', 'bar': '/path/b/'} })

In [ ]:
db.all()